# Annulus: same distributions
* Let `P`, `Q` be two point clouds sampled from the same distribution on an annulus. 
* This notebook contains a comparison to existing methods (induced matching, cycle registration) and application of the analogous bars method.

### Outline
1. Load points
2. Application of induced matching and cycle registration
3. Application of similarity-centric analogous bars method.

In [1]:
using Revise
includet("../../../extension_method.jl")

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-9568269704571493396\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-10593419359199592768\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-9568269704571493396\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

┌ Warning: ORCA.jl has been deprecated and all savefig functionality
│ has been implemented directly in PlotlyBase itself.
│ 
│ By implementing in PlotlyBase.jl, the savefig routines are automatically
│ available to PlotlyJS.jl also.
└ @ ORCA /opt/julia/packages/ORCA/U5XaN/src/ORCA.jl:8


In [2]:
using .ext
using Distances
using Distributions
using Eirene
using Plots
using JLD
using Measures
using StatsBase

# 1. load points

In [3]:
# load points
saved = load("data.jld")
P = saved["P"]
Q = saved["Q"]
mean = saved["mean"]
variance = saved["variance"];

┌ Warning: JLD incorrectly extends FileIO functions (see FileIO documentation)
└ @ FileIO /opt/julia/packages/FileIO/JA3Vl/src/loadsave.jl:215


The points P and Q were generated according to the following code. The distance from the center of the annulus follows a normal distribution.

In [277]:
"""
# specify normal distributions mean and variance
mean = 1
variance = 0.3

# generate P
n_P = 20
P = zeros(n_P,2)
theta = rand(Uniform(0,2*pi), n_P)
distance = rand(Normal(mean, variance), n_P)
for i=1:n_P
    P[i, 1] = distance[i] * cos(theta[i]) 
    P[i, 2] = distance[i] * sin(theta[i])
end

# generate Q
n_Q = 20
Q = zeros(n_Q, 2)
theta = rand(Uniform(0,2*pi), n_Q)
distance = rand(Normal(mean, variance), n_Q)
for i=1:n_Q
    Q[i, 1] = distance[i] * cos(theta[i]) 
    Q[i, 2] = distance[i] * sin(theta[i])
end
"""

In [4]:
# plot points
plot_P_Q(P, Q)

<!DOCTYPE html>
 
 
 Plots.jl

# 2. Application of existing methods

## (a) compute barcodes on $P$, $Q$, and $P \cup Q$

In [5]:
# compute distances among points
union = vcat(P, Q)
D = Distances.pairwise(Euclidean(), transpose(union), transpose(union), dims=2)

# Define submatrices 
n_P = size(P)[1]

D_P = D[1:n_P, 1:n_P]
D_Q = D[n_P+1:end, n_P+1:end]
D_P_Q = D[1:n_P, n_P+1:end]
    # rows (landmarks): P
    # columns (witness) : Q
D_Q_P = D[n_P+1:end, 1:n_P];
    # rows (landmarks): Q
    # columns (witness) : P

In [6]:
# Run eirene & plot barcodes
C_P = eirene(D_P, record = "all")
C_Q = eirene(D_Q, record = "all")
C = eirene(D, record = "all");

In [7]:

# plot barcodes
l = grid(3,1)
p1 = plot_barcode(barcode(C_P, dim = 1), lw = 5, title = "Barcode(VR(P))", titlefontsize = 12)
p2 = plot_barcode(barcode(C, dim = 1) , lw = 5, title = "Barcode(VR(P U Q))", titlefontsize = 12)
p3 = plot_barcode(barcode(C_Q, dim = 1) , lw = 5, title = "Barcode(VR(Q))", titlefontsize = 12)
plot(p1, p2, p3, layout = l, size = (500, 700))

<!DOCTYPE html>
 
 
 Plots.jl

## (b) Induced matching / cycle registration  
* We have $VR(P) \rightarrow VR(P \cup Q) \leftarrow VR(Q)$ 
* To see the results of induced matching / cycle registration on this dataset, we need to find the induced matching between $VR(P) \rightarrow VR(P \cup Q)$, and $VR(Q) \rightarrow VR(P \cup Q)$ 
* To find induced matching for $f: VR(P) \rightarrow VR(P \cup Q)$, we need to compute the image persistence barcode $im(f)$. I'm not sure if there is a software for this, so for now, we'll just use a 'hack':
    * For an interval $I$ in $VR(P)$, find its cycle rep. Find the death time of this cycle in $VR(P \cup Q)$. If there exists an interval $I_* \in barcode(VR(P \cup Q))$, then induced matching will map $I$ to $I_*$. 
    * Similarly for induced matching via $g: VR(Q) \to VR(P \cup Q)$.

### (b)(i) Induced matching $ f: VR(P) \to VR(P \cup Q) $
* From the induced matching paper (2015) Theorem 6.1, if an interval $[b,d] \in barcode(VR(P))$ matches to $[b', d'] \in barcode(VR(P \cup Q))$, they must satisfy $b' \leq b < d' \leq d$.
* The unique interval (interval 1) of $barcode(VR(P))$ has a birth time of $b = 1.05$. In $barcode(VR(P \cup Q))$, the latest death time is at $0.93$. So all $d' < b$. Interval 1 of $barcode(VR(P))$ thus doesn't match to any intervals in $barcode(VR(P \cup Q))$. 

### (b)(ii) Induced matching $g: VR(P) \to VR(P \cup Q) $
* From the induced matching paper (2015) Theorem 6.1, if an interval $[b,d] \in barcode(VR(AL))$ matches to $[b', d'] \in barcode(VR(V1 \cup AL))$, they must satisfy $b' \leq b < d' \leq d$.
* So interval 1 of barcode(VR(AL)) can possible match to interval 2 or 3 of $barcode(VR(V1 \cup AL))$.
* To compute the induced matching between $barcode(VR(Q))$ and $barcode(VR(P \cup Q))$, we need to find the barcode of the image persistence $im (g)$. 
* I'm not sure if there's any software for computing the barcode of the image persistence. So I just checked for the death time of interval 5 in $VR(P \cup Q)$. 
* The following code computes the death time of cycle 5 from $VR(Q)$ in $VR(P \cup Q)$

In [8]:
S = classrep(C_Q, class = 5, dim = 1)

# cyclerep as [simplex1, simplex2, ..., simplexk], where each simplex is a list of vertices
chain_vertices = []
for j=1:size(S)[2]
    append!(chain_vertices, [sort(S[:,j])])
end

chain_vertices = [item + [n_P, n_P] for item in chain_vertices]
#chain = vertices_to_chains(chain_vertices, C, 1)
chain = ext.chain_to_index(chain_vertices, C)
Eirene.deathtime(C, chain = chain, dim = 1)

0.9295147917125612

From looking at the death time, we know that interval 5 of $VR(Q)$ matches to interval 5 of $VR(P \cup Q)$.

# 3. Implement the similarity-centric analogous bars method

In [9]:
# Compute Vietoris-Rips persistence on two regions
dim = 1
VR_P = eirene(D_P, record = "all", maxdim = dim)
VR_Q = eirene(D_Q, record = "all", maxdim = dim )

# compute Witness persistence
W_P = compute_Witness_persistence(D_P_Q, maxdim = dim)
W_Q = compute_Witness_persistence(D_Q_P, maxdim = dim);

In [10]:
# plot all four barcodes
l = grid(4,1)
p1 = plot_barcode(barcode(VR_P, dim = 1), lw = 3, title = "Barcode(VR(P))", titlefontsize = 12)
p2 = plot_barcode(barcode(W_P["eirene_output"], dim = 1) , lw = 3, title = "Barcode(W(P,Q))", titlefontsize = 12)
p3 = plot_barcode(barcode(W_Q["eirene_output"], dim = 1) , lw = 3, title = "Barcode(W(Q,P))", titlefontsize = 12)
p4 = plot_barcode(barcode(VR_Q, dim = 1), lw = 2, title = "Barcode(VR(Q))", titlefontsize = 12)
plot(p1, p2, p3, p4, layout = l, size = (500, 700))

<!DOCTYPE html>
 
 
 Plots.jl

In [11]:
# select interval
W_PQ_bar = 3

# run similarity-centric analogous bars method
extension_P, extension_Q = run_similarity_analogous(VR_P = VR_P, 
                                                    D_P = D_P, 
                                                    VR_Q = VR_Q, 
                                                    D_Q = D_Q, 
                                                    W_PQ = W_P, 
                                                    W_PQ_bar = W_PQ_bar, 
                                                    dim = dim);

In [12]:
plot_analogous_bars(extension_P, extension_Q)

<!DOCTYPE html>
 
 
 Plots.jl